```{contents}
```
## **Supervisor Agent in LangGraph**

A **Supervisor Agent** in LangGraph is a **control agent** responsible for **planning, coordination, task delegation, and convergence** of a multi-agent system.
It transforms independent agents into a **coherent, goal-directed architecture**.

---

### **1. Core Intuition**

In complex problems, no single agent should control everything.
Instead, LangGraph uses:

> **One supervisor → many workers → controlled convergence**

The **Supervisor Agent**:

* Understands the **global objective**
* Decomposes work into **subtasks**
* Assigns tasks to **specialized agents**
* Monitors progress
* Resolves conflicts
* Produces the **final decision**

---

### **2. Supervisor Responsibilities**

| Responsibility      | Description                      |
| ------------------- | -------------------------------- |
| Global Planning     | Build overall execution plan     |
| Task Decomposition  | Split problem into subtasks      |
| Delegation          | Route subtasks to workers        |
| Scheduling          | Control execution order          |
| Monitoring          | Track agent outputs              |
| Verification        | Validate intermediate results    |
| Conflict Resolution | Resolve contradictory outputs    |
| Convergence         | Decide when system is done       |
| Memory Management   | Maintain shared state            |
| Safety Control      | Enforce policies and constraints |

---

### **3. Conceptual Architecture**

```
                User Goal
                    |
              ┌─────────────┐
              │ Supervisor  │
              └──────┬──────┘
       ┌─────────────┼─────────────┐
       ↓             ↓             ↓
  Research Agent  Compute Agent  Critic Agent
       ↓             ↓             ↓
       └─────────────┴─────────────┘
                 Merge & Decide
```

---

### **4. State Design**

```python
class State(TypedDict):
    task: str
    plan: str
    research: str
    computation: str
    critique: str
    decision: str
```

The **Supervisor writes and reads every field**, giving it global visibility.

---

### **5. LangGraph Implementation**

#### **Worker Agents**

```python
def research_agent(state):
    return {"research": f"Research on {state['task']}"}

def compute_agent(state):
    return {"computation": "Computed result"}

def critic_agent(state):
    return {"critique": "Looks valid"}
```

#### **Supervisor Node**

```python
def supervisor(state):
    if not state.get("plan"):
        return {"plan": "Research → Compute → Critique"}
    if not state.get("decision") and state["critique"]:
        return {"decision": "Final answer based on all agents"}
```

#### **Graph Construction**

```python
builder = StateGraph(State)

builder.add_node("supervisor", supervisor)
builder.add_node("research", research_agent)
builder.add_node("compute", compute_agent)
builder.add_node("critic", critic_agent)

builder.set_entry_point("supervisor")

builder.add_edge("supervisor", "research")
builder.add_edge("research", "compute")
builder.add_edge("compute", "critic")
builder.add_edge("critic", "supervisor")

builder.add_conditional_edges("supervisor",
    lambda s: END if s.get("decision") else "research",
    {"research": "research", END: END}
)
```

---

### **6. Execution Behavior**

```
Supervisor → Research → Compute → Critic → Supervisor → ... → END
```

The system **loops** until the supervisor decides completion.

---

### **7. Why Supervisor Agent Matters**

| Without Supervisor   | With Supervisor      |
| -------------------- | -------------------- |
| Rigid pipelines      | Adaptive systems     |
| No global control    | Central coordination |
| No conflict handling | Conflict resolution  |
| Hard to scale        | Easily extensible    |
| Fragile workflows    | Robust systems       |

---

### **8. Variants of Supervisor Design**

| Variant                 | Use Case            |
| ----------------------- | ------------------- |
| Static Supervisor       | Fixed workflow      |
| Dynamic Supervisor      | Runtime planning    |
| Hierarchical Supervisor | Multi-level control |
| Distributed Supervisors | Large-scale systems |
| Human-Supervised        | Regulated domains   |

---

### **9. Production Use Cases**

* Enterprise decision systems
* Autonomous research agents
* Trading & risk analysis
* Legal & compliance review
* Cybersecurity monitoring
* Operations automation

---

### **10. Mental Model**

The Supervisor Agent acts as the **operating system kernel** of your LangGraph:

> **Allocates work → manages state → ensures correctness → decides termination**



### Demonstration

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

# -------------------- STATE --------------------

class State(TypedDict, total=False):
    task: str
    plan: str
    research: str
    computation: str
    critique: str
    decision: str
    step: int

# -------------------- WORKER AGENTS --------------------

def research_agent(state: State):
    return {"research": f"Key facts about: {state['task']}", "step": state["step"] + 1}

def compute_agent(state: State):
    return {"computation": f"Analysis based on: {state['research']}", "step": state["step"] + 1}

def critic_agent(state: State):
    return {"critique": f"Quality check passed for: {state['computation']}", "step": state["step"] + 1}

# -------------------- SUPERVISOR --------------------

def supervisor(state: State):
    if "plan" not in state:
        return {"plan": "Research → Compute → Critique", "step": state["step"] + 1}
    if state["step"] >= 4:
        return {"decision": f"Final answer for '{state['task']}'", "step": state["step"] + 1}
    return {"step": state["step"]}

# -------------------- BUILD GRAPH --------------------

builder = StateGraph(State)

builder.add_node("supervisor", supervisor)
builder.add_node("research", research_agent)
builder.add_node("compute", compute_agent)
builder.add_node("critic", critic_agent)

builder.set_entry_point("supervisor")

# Removed unconditional edge - let conditional edge handle routing
builder.add_edge("research", "compute")
builder.add_edge("compute", "critic")
builder.add_edge("critic", "supervisor")

builder.add_conditional_edges(
    "supervisor",
    lambda s: END if "decision" in s else "research",
    {"research": "research", END: END},
)

graph = builder.compile()

# -------------------- RUN --------------------

result = graph.invoke({"task": "Impact of AI on education", "step": 0})

print("=== Supervisor Agent Execution ===\n")
print(f"Task: {result['task']}")
print(f"Plan: {result['plan']}")
print(f"Research: {result['research']}")
print(f"Computation: {result['computation']}")
print(f"Critique: {result['critique']}")
print(f"Decision: {result['decision']}")
print(f"Total steps: {result['step']}")

=== Supervisor Agent Execution ===

Task: Impact of AI on education
Plan: Research → Compute → Critique
Research: Key facts about: Impact of AI on education
Computation: Analysis based on: Key facts about: Impact of AI on education
Critique: Quality check passed for: Analysis based on: Key facts about: Impact of AI on education
Decision: Final answer for 'Impact of AI on education'
Total steps: 5
